In [285]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [286]:
cd drive/MyDrive

[Errno 2] No such file or directory: 'drive/MyDrive'
/content/drive/MyDrive


In [287]:
!pip install transformers
!pip install sentencepiece
!pip install bertopic

In [288]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
import os
import json

In [289]:
import re

cond = re.compile('[가-힣]+')

def get_data(gallery):
  result = []
  for name in os.listdir(path='/content/drive/MyDrive/jolnon/' + gallery):
    with open('/content/drive/MyDrive/jolnon/' + gallery + '/' + name, 'r', encoding='utf-8') as f:
      data = json.load(f)
      valid = []
      for d in data['content'].split('\n'):
        res = []
        for word in d.split(' '):
          if cond.match(word) != None:
            res.append(word)
        d = ' '.join(res)
        if d and not d.replace(' ', '').isdecimal():
          valid.append(d)
      result.append('\n'.join([data['title']] + valid))
  return result

In [290]:
import os
import json
import re
from zipfile import ZipFile

cond = re.compile('[가-힣]+')

def get_data_from_zip(zip_name):
    zip_file_path = '/content/drive/MyDrive/jolnon/' + zip_name + '.zip'
    extract_path = '/content/' + zip_name

    # zip 파일 압축 해제
    with ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    result = []
    for name in os.listdir(path=extract_path):
        file_path = os.path.join(extract_path, name)
        if os.path.isdir(file_path):
            for name in os.listdir(path=file_path):
                with open(file_path + '/' + name, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    valid = []
                    for d in data['content'].split('\n'):
                        res = []
                        for word in d.split(' '):
                            if cond.match(word) != None:
                                res.append(word)
                        d = ' '.join(res)
                        if d and not d.replace(' ', '').isdecimal():
                            valid.append(d)
                    result.append('\n'.join([data['title']] + valid))
        else:
            with open(extract_path + '/' + name, 'r', encoding='utf-8') as f:
                data = json.load(f)
                valid = []
                for d in data['content'].split('\n'):
                    res = []
                    for word in d.split(' '):
                        if cond.match(word) != None:
                            res.append(word)
                    d = ' '.join(res)
                    if d and not d.replace(' ', '').isdecimal():
                        valid.append(d)
                result.append('\n'.join([data['title']] + valid))
    return result


In [291]:
import random
random.seed(42)
galleries = ['giants_new2','cock_tail']#,'ktwiz','kartriderdrift','skwyverns_new1','ncdinos','samsunglions_new','doosanbears_new1','giants_new2','sh_new','lgtwins_new','tigers_new']
dataset = []
for e in galleries:
  dataset.append([get_data_from_zip(e)])
cocktail = []
for data in dataset:
  for sen in data:
    cocktail+=sen

In [292]:
def remove_words(sentence, word):
    return ' '.join([w for w in sentence.split(' ') if word not in w.lower()])
stopwords = []#['ㅋㅋ', 'ex', '나는', 'on', '이미지', '순서', '오늘', '일단', 'and', '이야', '그리고', '내일', '그냥', '000', '조금', '살짝', 'ㅇㅇ', 'ㅈㄱㄴ', '있음', '이거', '내가', '칵하하하', '칵하핫', '근데', '지듣노', 'youtube ', '야스중', '우흥', '한다', 'ㅎㅎ', 'ㅠㅠ', '로오오오오오옹', '하고', '아침', '것도', '추천', '혹시', '새낀데', '같다']
for i in range(len(cocktail)):
  for j in range(len(stopwords)):
    cocktail[i] = remove_words(cocktail[i],stopwords[j])

In [293]:
import collections

def remove_top_n_words(data, n):
    words = [word for sentence in data for word in sentence.split(' ')]
    word_count = collections.Counter(words)
    top_n_words = [word for word, _ in word_count.most_common(n)]
    for i in range(len(data)):
        for word in top_n_words:
            if word=='':
                continue
            data[i] = remove_words(data[i], word)
    return data

def remove_empty_sentences(cocktail, real_labels):
  indices = [i for i, sentence in enumerate(cocktail) if sentence.strip() != '']
  cocktail = [cocktail[i] for i in indices]
  real_labels = [real_labels[i] for i in indices]
  return cocktail, real_labels


In [357]:
real_labels = []
for i in range(len(dataset)):
  real_labels += [i] * len(dataset[i][0])
cocktail = remove_top_n_words(cocktail, 0)
cocktail, real_labels = remove_empty_sentences(cocktail, real_labels)

In [358]:
from transformers import AutoModel, AutoTokenizer
from umap import UMAP

# umap_model = UMAP(random_state=42)
# umap_model = UMAP(n_neighbors=15, n_components=10, metric='cosine', random_state=42)
model = AutoModel.from_pretrained("klue/roberta-large")
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [359]:
# bertopic_model = BERTopic(language='multilingual',
#                           # umap_model=umap_model,
#                           nr_topics=100,
#                           top_n_words=1)
# topics, probs = bertopic_model.fit_transform(cocktail)

In [360]:
# bertopic_model.visualize_topics()
# bertopic_model.visualize_distribution(probs[0])

In [361]:
# from transformers import RobertaModel, RobertaTokenizer
# import torch

# model = RobertaModel.from_pretrained("klue/roberta-large")
# tokenizer = RobertaTokenizer.from_pretrained("klue/roberta-large")

In [362]:
class CustomTokenizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
  # def __call__(self, target):
  #   return self.tokenizer.tokenize(target)
  def __call__(self, target):
    tokens = self.tokenizer.tokenize(target)
    if isinstance(tokens[0], str):
      indices = [i for i, token in enumerate(tokens) if not token.startswith('##')]
      tokens = [tokens[i] for i in indices]
    elif isinstance(tokens[0][0], str):
      for j in range(len(tokens)):
        indices = [i for i, token in enumerate(tokens[j]) if not token.startswith('##')]
        tokens[j] = [tokens[j][i] for i in indices]
    return tokens

In [363]:
from sklearn.feature_extraction.text import TfidfVectorizer

custom_tokenizer = CustomTokenizer(tokenizer)
# vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [364]:
# bertopic_model_embedding = BERTopic(embedding_model=model,
#                                     # umap_model=umap_model,
#                                     nr_topics=100,
#                                     top_n_words=1)
# topics, probs = bertopic_model_embedding.fit_transform(cocktail)

In [365]:
# bertopic_model_tokenizer = BERTopic(vectorizer_model=vectorizer,
#                                     # umap_model=umap_model,
#                                     nr_topics=100,
#                                     top_n_words=1)
# topics, _ = bertopic_model_tokenizer.fit_transform(cocktail)

In [366]:
bertopic_model_embed_token = BERTopic(embedding_model=model,
                                      # umap_model=umap_model,
                                      vectorizer_model=vectorizer,
                                      nr_topics=100,
                                      top_n_words=3)
topics, _ = bertopic_model_embed_token.fit_transform(cocktail)

Token indices sequence length is longer than the specified maximum sequence length for this model (61889 > 512). Running this sequence through the model will result in indexing errors


In [367]:
topic_info = bertopic_model_embed_token.get_topic_info()
number_of_topics = len(topic_info) - 1
print(f"생성된 주제의 수: {number_of_topics}")
lst = []
for index, row in topic_info.iterrows():
    topic_num = row['Topic']
    if topic_num != -1:
        topic_name = row['Name']
        pure_topic_name = "_".join(topic_name.split("_")[1:])
        lst.append(pure_topic_name)
for i in lst:
  print(i)

생성된 주제의 수: 71
뿌_딱_아깝
릴_내_베르
ㄴ_^_ㄱ
a_강민호_o
+_❤_근
j_현역_통산
생과_인_벌써
c_앱_질렀
봐라_병신_이거
vs_칵_핫
9_10_화력
7_5_2
쓱_싸_쓰레기
송도_인천_도시
손주_날려_좌회전
경상도_뒤통수_베일
털어_,_서브
검지_방금_버스
동맹_갸_홍
끓인_입_오늘
등록_구분_쿠폰
나부_하자_시작
용_여긴_조기
그란_자이언츠_니트
쥐_근무_우리
인증_에비_연락
릴_받_이미지
리필_스트로_피츠
·_다이_09
바질_이미지_순서
시대_범위_친구
시그_._산다
입_시험_압
불꽃_진지_뿌려
롬_~_최강
홈런_?_함
지리_임마_아들
할머니_달린다_자
시작_야스_황성
에반_윌리엄스_릴
나눔_만_비트
나눔_깜_소독
나눔_받_키위
붙_역_쉐
소다_베르_릴
단_목_돈
어제_떴_센
끊_공간_앞
모집_아니야_비호
생수_<_과학
워_~_뭐
능력_슬슬_유치
뭐_아무_시키
건강_낮잠_선물
곰팡이_견과_종
소금_생겼_병
통풍_플로_대통령
쳐_박세_최악
한다_결과물_올라온
선생_터뜨리_인정
합격_옮기_라인업
모스_믹스_헤이
?_최강_하나
빨대_실수_코스
존나_매년_바를
곤_받_드
코코넛_파인애플_그래
서_천_<
안치_농협_노무현
아닐까_화이트_다크
플_생크림_오크


In [368]:
from bertopic.vectorizers import ClassTfidfTransformer
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
bertopic_model_embed_token = BERTopic(embedding_model=model,
                                      # umap_model=umap_model,
                                      vectorizer_model=vectorizer,
                                      ctfidf_model=ctfidf_model,
                                      nr_topics=100,
                                      top_n_words=3,
                                      calculate_probabilities=True)
topics, _ = bertopic_model_embed_token.fit_transform(cocktail)

In [369]:
topic_info = bertopic_model_embed_token.get_topic_info()
number_of_topics = len(topic_info) - 1
print(f"생성된 주제의 수: {number_of_topics}")
lst = []
for index, row in topic_info.iterrows():
    topic_num = row['Topic']
    if topic_num != -1:
        topic_name = row['Name']
        pure_topic_name = "_".join(topic_name.split("_")[1:])
        lst.append(pure_topic_name)
for i in lst:
  print(i)

생성된 주제의 수: 72
뿌_아깝_취한다
호두_귀_삶
a_강민호_o
ㄴ_^_엌
+_❤_근
j_통산_현역
생과_인_비싸
c_질렀_멤버십
vs_빨대_핫
봐라_템_5000
7_5_700
9_10_화력
쓱_싸_쓰레기
천하_기운_권
끓인_뜻_청량
송도_우리나라_쇼핑
손주_날려_좌회전
경상도_뒤통수_바다
털어_서브_글라스
나부_하자_@
여긴_조기_절반
면세_시대_범위
엮_옮겨_조회
구분_등록_을
윌리엄스_에반_려
바질_타임_사마
어이_오일_쪼
홈런_이택_삭제
롬_사이드_완료
은퇴_화장실_연락
그란_자이언츠_”
붙_역_밝
끊_앞_옥수수
모집_고백_과제
소독_깜_독
야스_부스_수확
치우_근무_찾아
18_잤_땜
지리_임마_아들
비트_맨해튼_만
키위_나눔_대학생
소다_베르_케
·_09_가리
스트로_리필_피츠
불꽃_뿌려_엑스포
목_무시_잠들
센_떴_구장
능력_다르_유치
생수_셀_가치
구도_가장_익숙
최정_이대호_오해
원두_뭔가_골드
워_이진_관
고장_냉동실_동네
신청_망하_파도
감사_생겼_관중
오타_그랬_정도
한다_결과물_올라온
실수_김해_제출
선발투수_펜_통풍
매년_자기_시가
바를_진로_대단
플레_견과_곰팡이
부심_신인_맹
금액_굿_중요
상큼_검색_흐르
품절_생크림_카카오
에_멘_메
모스_헤이_탓
분류_곤_왜냐면
옮기_합격_연기
안치_응_킹


In [370]:
table = [[0 for r in range(2)] for t in range(number_of_topics)]

for j in range(number_of_topics):
  for i in range(len(topics)):
    if topics[i]==j:
      table[j][real_labels[i]]+=1
print(table)

[[14, 6], [1, 25], [15, 1], [16, 8], [18, 0], [54, 1], [50, 117], [0, 13], [3, 67], [14, 4], [24, 10], [24, 7], [23, 20], [7, 4], [0, 13], [15, 0], [27, 32], [17, 22], [85, 151], [1, 11], [17, 7], [16, 8], [13, 7], [2, 11], [1, 19], [0, 14], [3, 44], [24, 20], [10, 18], [43, 9], [13, 10], [12, 5], [15, 9], [19, 9], [3, 20], [6, 18], [6, 4], [5, 8], [24, 7], [1, 34], [3, 77], [4, 20], [3, 19], [1, 16], [19, 38], [10, 14], [16, 13], [3, 9], [6, 6], [9, 9], [4, 9], [5, 6], [22, 42], [5, 8], [43, 38], [11, 11], [41, 66], [12, 12], [28, 22], [18, 24], [11, 7], [6, 7], [16, 8], [5, 5], [3, 27], [2, 9], [2, 33], [2, 12], [1, 9], [3, 46], [2, 10], [53, 68]]


In [ ]:
### Reducing outliers
# topics = bertopic_model_embed_token.reduce_outliers(cocktail, topics, strategy="c-tf-idf", threshold=0.1)
# topics = bertopic_model_embed_token.reduce_outliers(cocktail, topics, strategy="distributions")

In [ ]:
import matplotlib.pyplot as plt

colors = ["#0000FF", "#FF0000"]

data = [[] for _ in range(len(dataset))]
for j in range(len(dataset)):
    data[j] = [topics[i] for i in range(len(real_labels)) if real_labels[i] == j]
    plt.hist(data[j], bins=number_of_topics, color=colors[j], histtype='bar')
    plt.show()

plt.hist(data, bins=number_of_topics, color=colors, histtype='bar', density=True)
plt.show()